In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from statsmodels.stats.outliers_influence import variance_inflation_factor
from scipy.stats import entropy


from evidently.report import Report
from evidently.metric_preset import DataDriftPreset
from evidently.metrics import *

from tqdm import tqdm
tqdm.pandas()
import warnings


warnings.filterwarnings("ignore")

In [ ]:
model_output=r"D:\Pranto\Working\LSA and Writeoff\LSA and Writeoff New Model\Writeoff\DATA\DATA\Model Output"

name1=r"Writeoff Model Jan24_Feb25"

# Import Train and Test data

In [ ]:
path_main=r"D:\Pranto\Working\LSA and Writeoff\LSA and Writeoff New Model\Writeoff\DATA\DATA"

name=r"Writeoff model_jan24_Feb25_raw data with all variables V1_apply_binning V1"

In [ ]:
try:
    df=pd.read_csv(rf"{path_main}\{name}.csv")
except:
    df=pd.read_excel(rf"{path_main}\{name}.xlsx")

print(df.shape)

df.head(3)

In [ ]:
df['MONTH_YEAR_V1'].unique()

# Primary Key

In [ ]:
# df['Primary_Key']=df['CONNO'].astype(str)+"_"+df['MONTH'].astype(str)

# df.columns

In [ ]:
# df['MONTH'].unique()

In [ ]:
# df['Primary_Key'].unique()

In [ ]:
# # Export

# df.to_csv(rf"{path_main}\{name}_with Primary Key.csv",index=False)

In [ ]:
train_month=['JAN24', 'FEB24', 'MAR24', 'APR24', 'MAY24', 'JUN24', 'JUL24',
       'AUG24', 'SEP24', 'OCT24', 'NOV24', 'DEC24']
 
test_month=['JAN25','FEB25']
 
filter_column='MONTH_YEAR_V1'
 
target_variable='TARGET_VARIABLE'

columns_last_name='WoE'

In [ ]:
# train_month=['JUL23','AUG23','SEP23','OCT23','NOV23','DEC23','JAN24','FEB24','MAR24','APR24','MAY24','JUN24']

# test_month=['JUL24','AUG24','SEP24']

# filter_column='MONTH'

# target_variable='TARGET_VARIABLE_V1'

In [ ]:
# Train Data

train_data=df[df[filter_column].isin(train_month)]


X_train = train_data.filter(like=columns_last_name)


print("Shape of train Data====",train_data.shape,'\n',"Shape of X_train===",X_train.shape,'\n',"Unique month===",train_data[filter_column].unique())


print("===================================================")

# Create test data dictionary for CSI table
test_datasets = {}
# Test Data
for i in test_month:
    globals()[f'test_data_{i}']=df[df[filter_column].eq(i)]

    globals()[f'X_test_{i}'] = globals()[f'test_data_{i}'].filter(like=columns_last_name)

    print(f"Shape of test Data {i}====",globals()[f'test_data_{i}'].shape,'\n',f"Shape of X_test_{i}",globals()[f'X_test_{i}'].shape,'\n',"Unique month===",globals()[f'test_data_{i}'][filter_column].unique())


    


    # Add each test dataset to the dictionary
    test_datasets[f'Test_{i}'] = globals()[f'X_test_{i}']






In [ ]:
test_datasets

# CSI and VIF table

In [ ]:
def calculate_csi_by_value(train, test):
    """
    Calculate CSI for each unique value for each variable comparing train and test dataset.
    Returns a DataFrame with value-level details.
    """
    csi_details = []

    for column in train.columns:
        # Calculate count % for each unique value in train and test datasets
        train_value_counts = train[column].value_counts(normalize=True)
        test_value_counts = test[column].value_counts(normalize=True)

        # Align indices so both series have all unique values across train and test
        all_values = train_value_counts.index.union(test_value_counts.index)
        train_value_counts = train_value_counts.reindex(all_values).fillna(1e-10)
        test_value_counts = test_value_counts.reindex(all_values).fillna(1e-10)

        # Calculate CSI for each unique value
        value_csi = (train_value_counts - test_value_counts) * np.log(train_value_counts / test_value_counts)

        # Append data for each unique value
        for value in all_values:
            csi_details.append({
                'Variable': column,
                'Value': value,
                'Train %': train_value_counts[value],
                'Test %': test_value_counts[value],
                'Value CSI': value_csi[value]
            })

    return pd.DataFrame(csi_details)

def calculate_total_csi_and_decision(csi_df, threshold_stable=0.1, threshold_watch=0.25):
    """
    Calculate total CSI and decision based on total CSI for each variable.
    """
    total_csi_df = csi_df.groupby('Variable')['Value CSI'].sum().reset_index(name='Total CSI')

    # Determine stability decision
    total_csi_df['Decision'] = np.where(
        total_csi_df['Total CSI'] < threshold_stable, 'Stable',
        np.where(total_csi_df['Total CSI'] <= threshold_watch, 'On Watch', 'Unstable')
    )

    return total_csi_df

def generate_detailed_csi_table(train, test_datasets):
    """
    Generate a detailed CSI table with Total CSI and Decision columns for train and each test dataset.
    """
    detailed_csi_df = pd.DataFrame()
    # pass

    # Detailed CSI for train
    train_csi_details = calculate_csi_by_value(train, train)
    train_total_csi = calculate_total_csi_and_decision(train_csi_details)
    train_csi_details = train_csi_details.merge(train_total_csi, on='Variable')
    train_csi_details = train_csi_details.rename(columns={
        'Train %': 'Train %',
        'Total CSI': 'Train Total CSI',
        'Decision': 'Train Decision'
    })

    # Start with the train data in detailed format
    # detailed_csi_df = train_csi_details[['Variable', 'Value', 'Train %', 'Value CSI', 'Train Total CSI', 'Train Decision']]
    detailed_csi_df = train_csi_details[['Variable', 'Value', 'Train %']]
    
    # Detailed CSI for each test dataset
    for test_name, test_data in test_datasets.items():
        # Calculate CSI for each test dataset
        test_csi_details = calculate_csi_by_value(train, test_data)
        test_total_csi = calculate_total_csi_and_decision(test_csi_details)

        # Rename columns for the specific test dataset
        test_csi_details = test_csi_details.rename(columns={
            'Test %': f'{test_name} %',
            'Value CSI': f'{test_name} Value CSI'
        })
        test_total_csi = test_total_csi.rename(columns={
            'Total CSI': f'{test_name} Total CSI',
            'Decision': f'{test_name} Decision'
        })

        # Merge test bin details with total CSI and decision
        test_csi_details = test_csi_details.merge(test_total_csi, on='Variable')

        # Merge test details into the main detailed CSI DataFrame
        detailed_csi_df = detailed_csi_df.merge(test_csi_details[['Variable', 'Value', f'{test_name} %', f'{test_name} Value CSI', f'{test_name} Total CSI', f'{test_name} Decision']],
                                                on=['Variable', 'Value'],
                                                how='left')

    return detailed_csi_df





# Detailed CSI

In [ ]:
test_datasets

In [ ]:

# Generate the detailed CSI table
detailed_csi_table = generate_detailed_csi_table(X_train, test_datasets)

# Display the detailed CSI table
pd.set_option('display.max_columns', None)  # To show all columns
print("Detailed CSI Table:")
detailed_csi_table

# Variablewise Final decision of CSI

In [ ]:
# Assuming detailed_csi_df is your DataFrame
columns_with_decision = [col for col in detailed_csi_table.columns if 'decision' in col.lower()]

detailed_csi_table1=detailed_csi_table[['Variable']+columns_with_decision]

monthly_decision_table=detailed_csi_table1.drop_duplicates()

monthly_decision_table.head(3)

# Select those variables who are either stable or onwatch

In [ ]:
# Filter for variables that are either 'Stable' or 'On Watch' for all test months
stable_or_onwatch_vars = monthly_decision_table[
    monthly_decision_table.loc[:, monthly_decision_table.columns.str.endswith('Decision')].apply(
        lambda row: all(decision in ['Stable', 'On Watch'] for decision in row), axis=1)
]['Variable'].tolist()

print("Variables stable or on watch across all test months:")
stable_or_onwatch_vars


# Filter the Variables

In [ ]:


# Step 1: Calculating VIF
def calculate_vif(df):
    vif_df = pd.DataFrame()
    vif_df['Variable'] = df.columns
    vif_df['VIF'] = [variance_inflation_factor(df.values, i) for i in range(df.shape[1])]
    return vif_df

# Assuming X_train and stable_or_onwatch_vars are defined
X_train_V1 = X_train[stable_or_onwatch_vars]

# Step 2: Calculate VIF for X_train_V1
vif_df = calculate_vif(X_train_V1)
X_train_filtered = X_train_V1.loc[:, vif_df[vif_df['VIF'] < 5]['Variable']]

print("Done VIF calculation")

# Step 3: Filter columns for each test set in test_month list
# test_month = ['JUL24', 'AUG24', 'SEP24']  # Add any other months as needed
filtered_test_datasets = {}

for month in tqdm(test_month,desc="Processing Time"):
    # Access the test dataset dynamically
    X_test = globals()[f'X_test_{month}'][stable_or_onwatch_vars]
    filtered_test_datasets[f'X_test_{month}_filtered'] = X_test[X_train_filtered.columns]

# Print results for verification
print("Filtered Training Columns:======================", X_train_filtered.columns)




In [ ]:
# Merge Vif With Monthly CSI table

monthly_decision_table1=monthly_decision_table.merge(vif_df,on='Variable',how='left')

# Add the filter column based on VIF and Monthly Decision criteria
monthly_decision_table1['Variable Importance'] = np.where(
    (monthly_decision_table1['VIF'] < 5) & 
    monthly_decision_table1.loc[:, monthly_decision_table1.columns.str.endswith('Decision')].progress_apply(
        lambda row: all(decision in ['Stable', 'On Watch'] for decision in row), axis=1
    ),
    'Yes', 
    'No'
)

print(monthly_decision_table1['Variable Importance'].value_counts())
monthly_decision_table1.head(3)

# Export

In [ ]:
with pd.ExcelWriter(rf"{model_output}\{name1}_Detailed_CSI V1.xlsx",engine='xlsxwriter') as want:
    monthly_decision_table1.to_excel(want,sheet_name="Variable Wise CSI",index=False)
    detailed_csi_table.to_excel(want,sheet_name="Detailed CSI",index=False)
    vif_df.to_excel(want,sheet_name="VIF",index=False)

In [ ]:
detailed_csi_table